# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [40]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [41]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [42]:
%%capture --no-display

# Configure the map plot
city_data_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo=True, 
    size="Humidity", 
    color="Humidity", 
    tiles="OSM", 
    frame_width=700, 
    frame_height=500, 
    title="City Humidity Map"
)


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [43]:
# Example criteria: Temperature between 20°C and 30°C, Wind Speed < 10 m/s, Cloudiness < 50%
ideal_weather_df = city_data_df[
    (city_data_df["Max Temp"] >= 20) & 
    (city_data_df["Max Temp"] <= 30) & 
    (city_data_df["Wind Speed"] < 10) & 
    (city_data_df["Cloudiness"] < 50)
]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
7,7,carnarvon,-24.8667,113.6333,20.02,76,0,7.46,AU,1666108232
9,9,saint-pierre,-21.3393,55.4781,22.82,68,0,5.14,RE,1666108236
19,19,puerto ayora,-0.7393,-90.3518,22.98,82,41,1.79,EC,1666108242
20,20,avarua,-21.2078,-159.7750,22.03,73,20,2.57,CK,1666108220


### Step 3: Create a new DataFrame called `hotel_df`.

In [44]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""


# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,faya,SA,18.3851,42.4509,35,
7,carnarvon,AU,-24.8667,113.6333,76,
9,saint-pierre,RE,-21.3393,55.4781,68,
19,puerto ayora,EC,-0.7393,-90.3518,82,
20,avarua,CK,-21.2078,-159.7750,73,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [ ]:
# Set search radius to 10,000 meters
radius = 10000 

# Set base URL for the Geoapify API
base_url = "https://api.geoapify.com/v2/places"

# Print a message to follow up on the hotel search
print("Starting hotel search")

# Iterate through the DataFrame rows
for index, row in hotel_df.iterrows():
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Define the API request parameters
    params = {
        "categories": "accommodation.hotel",
        "filter": f"circle:{longitude},{latitude},{radius}",
        "bias": f"proximity:{longitude},{latitude}",
        "limit": 1,
        "apiKey": geoapify_key
    }

    # Make the API request
    response = requests.get(base_url, params=params)
    results = response.json()

    # Extract the hotel name
    try:
        hotel_df.loc[index, "Hotel Name"] = results["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{row['City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()


Starting hotel search
faya - nearest hotel: No hotel found
carnarvon - nearest hotel: No hotel found
saint-pierre - nearest hotel: No hotel found
puerto ayora - nearest hotel: No hotel found
avarua - nearest hotel: No hotel found
cape town - nearest hotel: No hotel found
kapaa - nearest hotel: No hotel found
mogadishu - nearest hotel: No hotel found
hilo - nearest hotel: No hotel found
lasa - nearest hotel: No hotel found
vaini - nearest hotel: No hotel found
east london - nearest hotel: No hotel found
banda - nearest hotel: No hotel found
atuona - nearest hotel: No hotel found
hermanus - nearest hotel: No hotel found
nalut - nearest hotel: No hotel found
makakilo city - nearest hotel: No hotel found
saint-philippe - nearest hotel: No hotel found
palencia - nearest hotel: No hotel found
inhambane - nearest hotel: No hotel found
sao filipe - nearest hotel: No hotel found
ambon - nearest hotel: No hotel found
port elizabeth - nearest hotel: No hotel found
georgetown - nearest hotel: No h

,City,Country,Lat,Lng,Humidity,Hotel Name
0,faya,SA,18.3851,42.4509,35,No hotel found
7,carnarvon,AU,-24.8667,113.6333,76,No hotel found
9,saint-pierre,RE,-21.3393,55.4781,68,No hotel found
19,puerto ayora,EC,-0.7393,-90.3518,82,No hotel found
20,avarua,CK,-21.2078,-159.7750,73,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [46]:
%%capture --no-display

# Configure the map plot with hover information
final_map = hotel_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo=True, 
    size="Humidity", 
    color="Humidity", 
    tiles="OSM", 
    frame_width=700, 
    frame_height=500, 
    title="Ideal Vacation Spots with Nearby Hotels",
    hover_cols=["City", "Country", "Hotel Name"]
)

# Display the map
final_map


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Country,Hotel Name)